### Import libraries

In [1]:
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from osgeo import ogr
import geopandas as gpd
import openpyxl
import seaborn as sns
import json
import os
import re
from datetime import datetime

### Insert files' path

In [2]:
# Define the year variable of the data
year = 2022

# Percent threshold for cleaning the AVR incomplete sensor data
percent_thresh = 75

# Assumed Harvest Index value for potatoes
HI = 0.75

# Dictionary for Dry Matter content per potato variety
dic_DM = {
  "Fontane": 0.23,
  "Ivory Russet": 0.21,
  "Challenger": 0.221,
    "Zorba" : 0.233,
    'Edison' : 0.214
}
dic_DM

# Input files paths
# Actual combined harvester potato field data from AVR connect portal
AVR_field = "/home/ucfaya1@ad.ucl.ac.uk/Downloads/HI_Data_yara_new/code_data/AVR_data/allFields_AVR_2022.xlsx"

# Geometry shapefile path
geometry_fields = "/home/ucfaya1@ad.ucl.ac.uk/Downloads/order-2449-20230803-075843/order-2449-20230803-075843.shp"

# Output files paths
csv_output_file_name = 'AVR_geometry{}.csv'.format(year)
csv_output_file_path = "/home/ucfaya1@ad.ucl.ac.uk/Downloads/HI_Data_yara_new/code_data/save_code_data/" + csv_output_file_name

## AVR actual yield dataset

In [3]:
# Read the Excel file into a DataFrame
df_AVR = pd.read_excel(AVR_field)
selected_columns = df_AVR.loc[:,['FieldName','Variety','StartDate','AmountHectareDone','AmountYieldTonsDone','AmountHectareDonePercentage','AverageTonsHa']]
df_AVR_Selec=df_AVR.drop(df_AVR.columns.difference(selected_columns.columns), axis=1)

# Apply threshold to filter out remove AVR incomplete field data
df_AVR_cleaned = df_AVR_Selec.drop(df_AVR_Selec[df_AVR_Selec['AmountHectareDonePercentage'] <= percent_thresh].index)

# Remove numbers and () to match the field names
pattern = r'[0-9,()]+'
df_AVR_cleaned['FieldName'] = df_AVR_cleaned['FieldName'].str.replace(pattern, '',regex=True)

# Remove space from the end of the string in field name
df_AVR_cleaned['FieldName'] = df_AVR_cleaned['FieldName'].str.strip()

# replace FO values in varieties with fontane
df_AVR_cleaned['Variety'] = df_AVR_cleaned['Variety'].replace('FO','Fontane')

In [4]:
# Unit conversion tons to kg by Multiplying by 1000 
df_AVR_cleaned['Average_kgha'] = df_AVR_cleaned['AverageTonsHa'] * 1000 # Average wet actual weight per ha
df_AVR_cleaned['Wet_Actual_Yield_kgfield'] = df_AVR_cleaned['AmountYieldTonsDone'] * 1000 # Total wet actual weight per field

# Create a new column with DM of each planted varieties per field
df_AVR_cleaned['DM'] = df_AVR_cleaned['Variety'].map(dic_DM)

# Wet to dry weight by multiplying it by Dry Matter content of corresponding planted variety
df_AVR_cleaned['dry_actual_average_kgha'] = df_AVR_cleaned['Average_kgha'] * df_AVR_cleaned['DM'] # Average dry actual weight per ha
df_AVR_cleaned['dry_actual_kgfield'] = df_AVR_cleaned['Wet_Actual_Yield_kgfield'] * df_AVR_cleaned['DM'] # Total dry actual weight per field

# Rename columns
df_AVR_cleaned = df_AVR_cleaned.rename(columns={'AmountYieldTonsDone': 'Wet_Actual_Yield_tonsfield'})
df_AVR_cleaned = df_AVR_cleaned.rename(columns={'AmountHectareDonePercentage': 'HaDone%'})
df_AVR_cleaned = df_AVR_cleaned.rename(columns={'StartDate': 'HarvestDate'})
df_AVR_cleaned.head()

,FieldName,HarvestDate,Variety,AmountHectareDone,HaDone%,Wet_Actual_Yield_tonsfield,AverageTonsHa,Average_kgha,Wet_Actual_Yield_kgfield,DM,dry_actual_average_kgha,dry_actual_kgfield
0,jacob lommelsendijk,2022-10-05 09:14:43,Fontane,1.340577,94,42.590538,31.770298,31770.297740,42590.53800,0.23,7307.168480,9795.823740
1,paul nijs achter heidy,2022-10-20 14:47:51,Fontane,1.117199,95,81.884590,73.294573,73294.573373,81884.59040,0.23,16857.751876,18833.455792
2,war gijs avoortstraat voor,2022-09-24 13:18:07,Fontane,2.071757,97,140.824182,67.973316,67973.315843,140824.18205,0.23,15633.862644,32389.561872
3,gutte zuiver station,2022-09-28 15:06:37,Fontane,1.580990,93,88.415033,55.923857,55923.857295,88415.03285,0.23,12862.487178,20335.457556
4,bart nijs bieman,2022-10-11 07:26:20,Fontane,1.574989,97,116.881572,74.211055,74211.055180,116881.57230,0.23,17068.542691,26882.761629


In [5]:
# Check for duplicate values in the 'FieldName' column
dup = df_AVR_cleaned['FieldName'].duplicated()
num_dup = dup.sum()
print(f"Number of duplicate field names: {num_dup}")

# Print the duplicate field names 
if num_dup > 0:
    print("Duplicate field names:")
    print(df_AVR_cleaned[df_AVR_cleaned['FieldName'].duplicated(keep=False)])

# Remove duplicates
df_AVR_cleaned = df_AVR_cleaned.drop_duplicates(subset=['FieldName'], keep='first')

Number of duplicate field names: 1
Duplicate field names:
          FieldName         HarvestDate  Variety  AmountHectareDone  HaDone%  \
14  cools dijckmans 2022-09-26 07:10:53  Fontane          12.893239       95   
15  cools dijckmans 2022-09-29 15:08:39  Fontane           5.203086       94   

    Wet_Actual_Yield_tonsfield  AverageTonsHa  Average_kgha  \
14                  737.386921      57.191750  57191.749792   
15                  296.106743      56.909828  56909.827800   

    Wet_Actual_Yield_kgfield    DM  dry_actual_average_kgha  \
14              737386.92055  0.23             13154.102452   
15              296106.74265  0.23             13089.260394   

    dry_actual_kgfield  
14       169598.991726  
15        68104.550810  


## Field Geometry

In [6]:
# Read the boundaries shapefile into a geopanda dataframe
field_basics_shp = gpd.read_file(geometry_fields)
column_headers = list(field_basics_shp.columns)

# Filter rows where 'crop' column has value 101 (reference for potato in IrriWatch database)
field_basics_shp = field_basics_shp[field_basics_shp['crop'] == 101]

# Remove numbers and () to match the field names and sapace at the end of the string
pattern = r'[0-9,()]+'
field_basics_shp['name'] =field_basics_shp['name'].str.replace(pattern, '',regex=True)
field_basics_shp = field_basics_shp.rename(columns={'name': 'FieldName'})
field_basics_shp = field_basics_shp.rename(columns={'soil_name_': 'SoilType'})

# Check for duplicate FieldName'
duplicate_fieldnames = field_basics_shp['FieldName'].duplicated()
num_duplicate_fieldnames = duplicate_fieldnames.sum()
print(f"Number of duplicate field names: {num_duplicate_fieldnames}")

# Print the duplicate field names
if num_duplicate_fieldnames > 0:
    print("Duplicate field names:")
    print(field_basics_shp[field_basics_shp['FieldName'].duplicated(keep=False)])
    
# Remove duplicates
field_basics_shp = field_basics_shp.drop_duplicates(subset=['FieldName'], keep='first')

Number of duplicate field names: 2
Duplicate field names:
      crop  irrigation               FieldName  soil  priority  ksat_bias  \
10   101.0           5        cools dijckmans    NaN         1          1   
164  101.0           5        cools dijckmans    NaN         1          1   
380  101.0           5  toon huijbregts heike    NaN         1          1   
512  101.0           5  toon huijbregts heike    NaN         1          1   

     eff_root_d  sat_soil_m  stress_typ  irrigati_1  irrigati_2    SoilType  \
10          NaN           1           3         NaN         NaN  Loamy Sand   
164         NaN           1           3         NaN         NaN  Loamy Sand   
380         NaN           1           3         NaN         NaN  Sandy Loam   
512         NaN           1           3         NaN         NaN  Sandy Loam   

     _id                                         crop_cycle  \
10    11  { "20220412": { "crop": "101", "stress_type": ...   
164  165  { "20220406": { "crop": 

In [7]:
# Merge AVR data with field boundary dataframe based on the 'FieldName' column
df_merged = df_AVR_cleaned.merge(field_basics_shp[['FieldName', 'geometry','SoilType']], on='FieldName', how='left')

# Check for incomplete data and remove the rows with nan values
nan_count = df_merged['geometry'].isna().sum()
print(f'NAN values count:{nan_count}')
df_merged.dropna(subset=['geometry'], inplace=True)
nan_count = df_merged['geometry'].isna().sum()

# Check for duplicate values in the 'FieldName' column
duplicate_fieldnames = df_merged['FieldName'].duplicated()
num_duplicate_fieldnames = duplicate_fieldnames.sum()
print(f"Number of duplicate field names: {num_duplicate_fieldnames}")

# Print the duplicate field names
if num_duplicate_fieldnames > 0:
    print("Duplicate field names:")
    print(df_merged[df_merged['FieldName'].duplicated(keep=False)])
    
# Remove duplicates 'FieldName' 
df_merged_no_duplicates = df_merged.drop_duplicates(subset=['FieldName'], keep='first')
df_merged_no_duplicates.head()

NAN values count:46
Number of duplicate field names: 0


,FieldName,HarvestDate,Variety,AmountHectareDone,HaDone%,Wet_Actual_Yield_tonsfield,AverageTonsHa,Average_kgha,Wet_Actual_Yield_kgfield,DM,dry_actual_average_kgha,dry_actual_kgfield,geometry,SoilType
4,bart nijs bieman,2022-10-11 07:26:20,Fontane,1.574989,97,116.881572,74.211055,74211.055180,116881.57230,0.23,17068.542691,26882.761629,"POLYGON ((5.11025 51.28476, 5.11017 51.28484, ...",Loamy Sand
5,bart nijs lukasstraat,2022-10-19 10:06:54,Fontane,2.979305,97,156.714842,52.601138,52601.138022,156714.84230,0.23,12098.261745,36044.413729,"POLYGON ((5.11603 51.23314, 5.11605 51.23315, ...",Sandy Loam
6,bart nijs naast war,2022-10-17 07:57:08,Fontane,0.697597,98,55.057347,78.924293,78924.292843,55057.34695,0.23,18152.587354,12663.189799,"POLYGON ((5.08139 51.24546, 5.08081 51.24638, ...",Sandy Clay Loam
7,bart nijs tussen war gijs,2022-10-17 10:11:51,Fontane,0.741455,99,63.163421,85.188423,85188.423067,63163.42070,0.23,19593.337305,14527.586761,"POLYGON ((5.07989 51.24665, 5.07993 51.24665, ...",Sandy Clay Loam
8,bart rommens molendijk eerste,2022-10-17 07:45:52,Fontane,0.902010,98,42.280780,46.873965,46873.965290,42280.77960,0.23,10781.012017,9724.579308,"POLYGON ((5.05993 51.23797, 5.05990 51.23798, ...",Sandy Clay Loam


In [8]:
df_merged_no_duplicates.columns

Index(['FieldName', 'HarvestDate', 'Variety', 'AmountHectareDone', 'HaDone%',
       'Wet_Actual_Yield_tonsfield', 'AverageTonsHa', 'Average_kgha',
       'Wet_Actual_Yield_kgfield', 'DM', 'dry_actual_average_kgha',
       'dry_actual_kgfield', 'geometry', 'SoilType'],
      dtype='object')

In [9]:
# Save as csv
df_merged_no_duplicates.to_csv(csv_output_file_path, index=False)

In [10]:
# Select only the 'FieldName' and 'geometry' columns
selected_columns = ['FieldName', 'geometry']
df_merged_no_duplicates = df_merged_no_duplicates[selected_columns]

# Save the edited GeoDataFrame as a shapefile
shp_output_file_name = 'geometry{}.shp'.format(year)
output_shapefile_path = "/home/ucfaya1@ad.ucl.ac.uk/Downloads/HI_Data_yara_new/code_data/save_code_data/" + shp_output_file_name
field_basics_shp.to_file(output_shapefile_path)

print("Shapefile saved successfully.")

Shapefile saved successfully.
